In [1]:
import torch
import torch.nn as nn
from torch import Tensor

import torchvision
from torchsummary import summary

from typing import Tuple,List
from torch import Tensor

from tqdm.auto import tqdm

import numpy as np

from typing import List

In [2]:
def print_size(tensor : Tensor):
  print(tensor.size())

# Intra dense block

In [3]:
# https://github.com/pytorch/vision/blob/master/torchvision/models/densenet.py
# https://github.com/prigoyal/pytorch_memonger/blob/master/tutorial/Checkpointing_for_PyTorch_models.ipynb

class DenseLayer(nn.Module):
  r"""Base dense layer where input is concatenated to the output (input forwarded into :param:base ).

  Given D_i-1 the input of i-th dense layer and D_i the output, 
  then D_i = [D_i-1, base(D_i-1)] and so D_i+1 = D_i and D_i+2 = [D_i+1,base(D_i+1)] therefore
  D_i+2 = [D_i, base(D_i+1)]
  So the output (D_i+2) is given by the recursive concatenation of all previous input and output (D_i = [ D_i-1, base(D_i-1) ] ) and the current output ( base(D_i+1) )

  Args:
    base : The base module used for creating the dense layer
  Attributes:
    base : The base module used for creating the dense layer
  """
  
  def __init__(self, base : nn.Module):
    super(DenseLayer,self).__init__()
    self.base = base

  def forward(self, input : Tensor):
    def concat_(input):
      return torch.cat([input,self.base(input)], dim=1)
    
    if input.requires_grad:
      return torch.utils.checkpoint.checkpoint(concat_,input)
    else:
      return concat_(input)

In [4]:
class IntraDenseBlock(nn.Module):
  """ Base Intra dense block

    Args:
      in_channels: Input channels of the block
      out_compressed_channels: Output channels of the input and output compression convolution at the beginning and at the end of the block
      intra_layer_output_features: Output channels of each convolution inside the block
      n_intra_layers: Number of convolutions inside the block
    """
  def __init__(self, in_channels : int , out_compressed_channels : int , intra_layer_output_features : int , n_intra_layers : int ):
    super(IntraDenseBlock,self).__init__()
    
    self.input_compression_layer = nn.Conv2d(in_channels=in_channels,
                                      out_channels=out_compressed_channels,
                                      kernel_size = 1, 
                                      stride = 1,
                                      padding = 0)
    
    self.output_compression_layer = nn.Conv2d(in_channels=out_compressed_channels + n_intra_layers * intra_layer_output_features,
                                      out_channels=out_compressed_channels,
                                      kernel_size = 1, 
                                      stride = 1,
                                      padding = 0)
    
    self.n_intra_layers = n_intra_layers

    list_intra_layers = list()    
    for n in range(n_intra_layers):
      # e.g:
      # n_intra_layers = 4
      # IC = input compression
      # OC = output compression
      # i:| = i-th Conv
      # IC -> 0:| -> 1:| -> 2:| -> 3:| -> [Here we have as input for OC: n_r + 4*n_g] OC 
      intra_layer_input_features = out_compressed_channels + n * intra_layer_output_features

      # A sequence of Conv(kernel_size=3,output_channels=n_g) and Relu
      list_intra_layers.append(
          DenseLayer(
              nn.Sequential(
                  nn.Conv2d(in_channels   = intra_layer_input_features,
                            out_channels  = intra_layer_output_features,
                            kernel_size = 3,
                            stride = 1,
                            padding = 1),
                  nn.ReLU(inplace=True))))
      
    self.intra_layers = nn.Sequential(*list_intra_layers)
  
  def forward(self, input : Tensor) -> Tensor:
    compressed_input = self.input_compression_layer(input)
    
    if compressed_input.requires_grad:
      # TODO: Replace 2 (segments) with a global variable
      dense_output = torch.utils.checkpoint.checkpoint_sequential(self.intra_layers, 2, compressed_input)
    else:
      dense_output = self.intra_layers(compressed_input)
    
    compressed_output = self.output_compression_layer(dense_output)

    return compressed_input + compressed_output

In [5]:
# TEST IntraDenseBlock


intra_dense_block = IntraDenseBlock(64,64,64,8).to("cuda:0")
summary(intra_dense_block,input_size = (64,48,48),batch_size=32, device='cuda')

intra_dense_block = IntraDenseBlock(64,32,32,4).to("cuda:0")
summary(intra_dense_block,input_size = (64,48,48),batch_size=32, device='cuda')


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [32, 64, 48, 48]           4,160
            Conv2d-2           [32, 64, 48, 48]          36,928
              ReLU-3           [32, 64, 48, 48]               0
        DenseLayer-4          [32, 128, 48, 48]               0
            Conv2d-5           [32, 64, 48, 48]          73,792
              ReLU-6           [32, 64, 48, 48]               0
        DenseLayer-7          [32, 192, 48, 48]               0
            Conv2d-8           [32, 64, 48, 48]         110,656
              ReLU-9           [32, 64, 48, 48]               0
       DenseLayer-10          [32, 256, 48, 48]               0
           Conv2d-11           [32, 64, 48, 48]         147,520
             ReLU-12           [32, 64, 48, 48]               0
       DenseLayer-13          [32, 320, 48, 48]               0
           Conv2d-14           [32, 64,

# Channel attention

In [6]:
class ChannelAttention(nn.Module):
  """Base Channel-Wise Attention module
  
    Args:
      in_channels: Input channels
      reduction: Reduction to use for compressing channels information ( channels_reduced = in_channels // reduction )
    """

  def __init__(self, in_channels : int , reduction : int):
    super(ChannelAttention,self).__init__()

    channels_reduced = in_channels // reduction

    
    self.attention = nn.Sequential(
        # max pooling
        nn.AdaptiveMaxPool2d(output_size=1),

        # apply reduction
        nn.Conv2d(in_channels=in_channels, out_channels=channels_reduced, kernel_size = 1),
    
        nn.ReLU(),

        # remove reduction
        nn.Conv2d(in_channels=channels_reduced, out_channels=in_channels, kernel_size = 1),

        nn.Sigmoid()           
    )

  def forward(self,input : Tensor) -> Tensor:
    attention_mask = self.attention(input)
    return input * attention_mask

In [7]:
#TEST ChannelAttention
CA = ChannelAttention(64,reduction = 16)

input_size = (64,48,48)

summary(CA,input_size, device="cpu")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
 AdaptiveMaxPool2d-1             [-1, 64, 1, 1]               0
            Conv2d-2              [-1, 4, 1, 1]             260
              ReLU-3              [-1, 4, 1, 1]               0
            Conv2d-4             [-1, 64, 1, 1]             320
           Sigmoid-5             [-1, 64, 1, 1]               0
Total params: 580
Trainable params: 580
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.56
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.57
----------------------------------------------------------------


# Dense Attention Block

In [8]:
def DenseAttentionBlock(in_channels : int , out_channels : int , intra_layer_output_features : int, n_intra_layers : int, reduction : int):
  '''Base Dense Attention Block used in the paper

  Args:
    in_channels : Input channels of the Dense Attention Block
    out_channels : Output channels of the Dense Attention Block ( which is equal to out_compressed_channels: output channels of the compression convolutions (at the beginning and at the end of IntraDenseBlock) ( see :class:`~IntraDenseBlock` ) )
    intra_layer_output_features: Output channels of each convolution inside the block  ( see :class:`~IntraDenseBlock` )
    n_intra_layers: Number of convolutions inside the block  ( see :class:`~IntraDenseBlock` )
    reduction: Reduction to use for compressing channels information ( channels_reduced = in_channels // reduction )  ( see :class:`~ChannelAttention` )
  '''
  
  return nn.Sequential(
      IntraDenseBlock(in_channels=in_channels, out_compressed_channels=out_channels, intra_layer_output_features=intra_layer_output_features, n_intra_layers=n_intra_layers),
      ChannelAttention(in_channels=out_channels, reduction = reduction)
      )

In [9]:
# Test DAB
DAB = DenseAttentionBlock(in_channels=64,out_channels=64,intra_layer_output_features=64,n_intra_layers=8,reduction=16)
summary(DAB,(64,48,48),batch_size = 32, device="cpu")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [32, 64, 48, 48]           4,160
            Conv2d-2           [32, 64, 48, 48]          36,928
              ReLU-3           [32, 64, 48, 48]               0
        DenseLayer-4          [32, 128, 48, 48]               0
            Conv2d-5           [32, 64, 48, 48]          73,792
              ReLU-6           [32, 64, 48, 48]               0
        DenseLayer-7          [32, 192, 48, 48]               0
            Conv2d-8           [32, 64, 48, 48]         110,656
              ReLU-9           [32, 64, 48, 48]               0
       DenseLayer-10          [32, 256, 48, 48]               0
           Conv2d-11           [32, 64, 48, 48]         147,520
             ReLU-12           [32, 64, 48, 48]               0
       DenseLayer-13          [32, 320, 48, 48]               0
           Conv2d-14           [32, 64,

# FMB

In [10]:
class FeatureMappingBranch(nn.Module):
  """ Feature Extraction Branch
  Args:
    in_channels: Input image channel(s) (1:grayscale, 3:rgb)
    low_level_features: Output channels of the low level features extraction convolution
    n_dab: Number of Dense Attention Blocks
    out_channels_dab: Output channels of the Dense Attention Block ( which is equal to out_compressed_channels: output channels of the compression convolutions (at the beginning and at the end of IntraDenseBlock) ( see :class:`~IntraDenseBlock` ) )
    intra_layers_output_features: Output channels of convolutions inside the Intra Dense Block ( see :class:`~IntraDenseBlock` )
    n_intra_layers: Number of convolutions inside Intra Dense Block ( see :class:`~IntraDenseBlock` )
    reduction: Reduction applied inside the Channel Attention layer ( see :class:`~ChannelAttention` )

  Attributes:
    final_output: the final output channels after the concatenation of all previous channels
  """
  def __init__(self, in_channels, low_level_features, n_dab, out_channels_dab, intra_layer_output_features, n_intra_layers, reduction):
    super(FeatureMappingBranch,self).__init__()
    
    self.final_output = low_level_features + n_dab * out_channels_dab

    # extract low level features from the input
    self.low_level_features = nn.Conv2d(in_channels=in_channels,
                                        out_channels=low_level_features,
                                        kernel_size = 3, 
                                        stride = 1,
                                        padding = 1)
    
    # Dense attention blocks
    self.n_dab = n_dab
    dabs = list()
    for n in range(n_dab):
      block_input_channels = low_level_features + n * out_channels_dab
      dabs.append(
          DenseLayer(
          DenseAttentionBlock(in_channels=block_input_channels,
                              out_channels=out_channels_dab,
                              intra_layer_output_features=intra_layer_output_features,
                              n_intra_layers=n_intra_layers,
                              reduction=reduction)))
    
    self.dabs = nn.Sequential(*dabs)
                                 
  def forward(self,input):
    low_level_features = self.low_level_features(input)
    if input.requires_grad:
      dense_output = torch.utils.checkpoint.checkpoint_sequential(self.dabs,2,low_level_features)
    else:
      dense_output = self.dabs(low_level_features)
    
    return dense_output

In [11]:
# Test FEB
FEB = FeatureMappingBranch(in_channels=64,low_level_features=64,n_dab=16,out_channels_dab=64,intra_layer_output_features=64,n_intra_layers=8,reduction=16)
summary(FEB,(64,48,48),device="cpu")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 48, 48]          36,928
            Conv2d-2           [-1, 64, 48, 48]           4,160
            Conv2d-3           [-1, 64, 48, 48]          36,928
              ReLU-4           [-1, 64, 48, 48]               0
        DenseLayer-5          [-1, 128, 48, 48]               0
            Conv2d-6           [-1, 64, 48, 48]          73,792
              ReLU-7           [-1, 64, 48, 48]               0
        DenseLayer-8          [-1, 192, 48, 48]               0
            Conv2d-9           [-1, 64, 48, 48]         110,656
             ReLU-10           [-1, 64, 48, 48]               0
       DenseLayer-11          [-1, 256, 48, 48]               0
           Conv2d-12           [-1, 64, 48, 48]         147,520
             ReLU-13           [-1, 64, 48, 48]               0
       DenseLayer-14          [-1, 320,

# IRB

In [12]:
class SpatialAttention(nn.Module):
  def __init__(self,in_channels, expansion ):
    super(SpatialAttention,self).__init__()
    
    self.sa = nn.Sequential(
        # expand the channels for each activation
        nn.Conv2d(in_channels=in_channels, out_channels=expansion * in_channels, kernel_size=1, stride=1,padding=0),

        # extract information at each spatial location
        nn.ReLU(),

        # restore the original channels dimension at each activation
        nn.Conv2d(in_channels=in_channels * expansion, out_channels=in_channels,kernel_size = 1,stride=1,padding=0),

        # create the spatial-attention mask
        nn.Sigmoid()
        )
  
  def forward(self,input):
    mask = self.sa(input)
    return input * mask

In [13]:
class ImageReconstructionBranch(nn.Module):
  """ Image Reconstruction Branch defined inside the paper 
  Args:
    in_channels: Input channels of the Image Reconstruction Branch ( which is the output channels of the Feature Extraction Branch )
    out_channels: Input image channel(s) (1:grayscale, 3:rgb)
    expansion: How much expand the channels inside the Spatial Attention for extracting high level frequency spatial information
  """
  def __init__(self, in_channels, out_channels, expansion):
    super(ImageReconstructionBranch,self).__init__()
    
    self.transform_into_image = nn.Conv2d(in_channels=in_channels,out_channels=out_channels,kernel_size=1,stride=1,padding=0)

    self.sa = SpatialAttention(out_channels, expansion)

  def forward(self, input_image, input_features):
    activations_to_image = self.transform_into_image(input_features)

    high_frequency_spatial_information = self.sa(activations_to_image)
    
    return input_image + high_frequency_spatial_information

In [14]:
low_level_features = 64
n_dab = 16
out_dab = 64

output_feature_extractor = low_level_features + n_dab * out_dab
image_channels = 3
image_size = 48
expansion = 2
scale = 1.5

# TODO: Interpolated before or after
import math
# interpolated_image = nn.functional.interpolate(input_image, scale_factor=self.scale,mode="bicubic", align_corners=False) 
interpolate_input_image_size = math.floor(image_size * scale)
interpolated_input = (3, interpolate_input_image_size, interpolate_input_image_size)

IRB = ImageReconstructionBranch(output_feature_extractor, image_channels,expansion)
summary(IRB,[interpolated_input,(output_feature_extractor,interpolate_input_image_size,interpolate_input_image_size)],batch_size = 32, device="cpu")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [32, 3, 72, 72]           3,267
            Conv2d-2            [32, 6, 72, 72]              24
              ReLU-3            [32, 6, 72, 72]               0
            Conv2d-4            [32, 3, 72, 72]              21
           Sigmoid-5            [32, 3, 72, 72]               0
  SpatialAttention-6            [32, 3, 72, 72]               0
Total params: 3,312
Trainable params: 3,312
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 10707552.00
Forward/backward pass size (MB): 30.38
Params size (MB): 0.01
Estimated Total Size (MB): 10707582.39
----------------------------------------------------------------


# ASDN

In [15]:
from collections import namedtuple
import math

Level = namedtuple("Level", "index scale size")

class LaplacianFrequencyRepresentation:
  def __init__(self, start,end,count, patch_size):
    
    self.count = count
    # if numpy is availablbe use np.linspace(start,end,count)
    step = (end - start) / (count - 1) 
    scales = [start + l*step for l in range(count)]
    sizes  = [math.floor(patch_size * scale) for scale in scales]
    self.information = [ Level(index,scale,size) for index,(scale,size) in enumerate(zip(scales,sizes)) ]

  def get_index(self, scale):
    """ Return the corresponding pyramid index for the given decimal scale
    Args:
      scale: Decimal scale
    Returns:
      index
    """
    return math.ceil((self.count - 1) * (scale - 1))

  def get_weight(self,scale):
    """ Return the interpolation weight for the given decimal scale
    Args:
      scale: Decimal scale
    Returns:
      weight
    """
    return (self.count - 1) * (self.information[self.get_index(scale)].scale - scale)

  def get_for(self,scale):
    index = self.get_index(scale)
    return self.information[index-1], self.information[index] 


In [16]:
lpr = LaplacianFrequencyRepresentation(1,2,11,48)
for info in lpr.information:
  print(info)

Level(index=0, scale=1.0, size=48)
Level(index=1, scale=1.1, size=52)
Level(index=2, scale=1.2, size=57)
Level(index=3, scale=1.3, size=62)
Level(index=4, scale=1.4, size=67)
Level(index=5, scale=1.5, size=72)
Level(index=6, scale=1.6, size=76)
Level(index=7, scale=1.7000000000000002, size=81)
Level(index=8, scale=1.8, size=86)
Level(index=9, scale=1.9, size=91)
Level(index=10, scale=2.0, size=96)


In [17]:
class ASDNNetwork(nn.Module):
  def __init__(self, pyramid_levels : int, input_image_channels : int, **FMB_kwargs):
    super(ASDNNetwork,self).__init__()
    """
    """

    # TODO: Rename FEB into FMB
    # TODO: Custom paramters for FeatureEXtractionBranch

    # Use custom parameters if available otherwise relay on ones defined in the paper
    self.feature_mapping_branch = FeatureMappingBranch(in_channels=FMB_kwargs.get("in_channels",3),
                                                          low_level_features=FMB_kwargs.get("low_level_features",64),
                                                          n_dab=FMB_kwargs.get("n_dab",16),
                                                          out_channels_dab=FMB_kwargs.get("out_compressed_channels",64),
                                                          intra_layer_output_features=FMB_kwargs.get("intra_layer_output_features",64),
                                                          n_intra_layers=FMB_kwargs.get("n_intra_layers",8),
                                                          reduction=FMB_kwargs.get("reduction",16))
    
    # change this if you are going to change the feature extractor
    self.in_channels_irb = self.feature_mapping_branch.final_output
    # and also this
    # expansion = 2 inside the paper [9]
    self.image_reconstruction_branches = nn.ModuleList([ImageReconstructionBranch(self.in_channels_irb,input_image_channels,2) for _ in range(pyramid_levels)])



  def forward(self, interpolated_patch, irb_index):
    features_extracted = self.feature_mapping_branch(interpolated_patch)
    
    output_leveli = self.image_reconstruction_branches[irb_index](interpolated_patch, features_extracted)
  
    return output_leveli

In [18]:
from functools import partial
lfr = LaplacianFrequencyRepresentation(1,2,11,48)
leveliminus1, leveli = lfr.get_for(scale = 1.3674)

#ASDN = ASDNNetwork(lfr.count,3, n_dab=8, n_intra_layers=8).to("cuda:0" if torch.cuda.is_available() else "cpu")
#ASDN.forward = partial(ASDN.forward, irb_index = leveli.index)
#summary(ASDN,(3,48,48),device="cuda" if torch.cuda.is_available() else "cpu" ,batch_size = 16)


# Test ASDN

In [21]:
from torch.utils.data import DataLoader,Dataset


cpu = "cpu"
gpu = "cuda:0"

scale = 1.3674
lfr = LaplacianFrequencyRepresentation(1,2,11,48)
leveliminus1, leveli = lfr.get_for(scale = scale)

class RandomTensorDataset:
  def __init__(self):
    self.patch = (3,48,48)
    self.len = 64
  
  def __getitem__(self, index):
    return torch.rand(*self.patch)
  
  def __len__(self):
    return self.len

# batch size 8 use 5.5GB 
loader = DataLoader(RandomTensorDataset(),batch_size=16)

ASDN = ASDNNetwork(lfr.count,3)
ASDN = ASDN.to(gpu)

start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)

ASDN.train()
for index,batch in enumerate(loader):

    # TODO: Move interpolation inside collate_fn
    start.record()
    interpolatedi = nn.functional.interpolate(batch,scale_factor=leveli.scale, mode="bicubic").to(gpu)
    outputi = ASDN(interpolatedi,irb_index=leveli.index).to(cpu)


    interpolatedi = nn.functional.interpolate(batch,scale_factor=leveliminus1.scale, mode="bicubic").to(gpu)
    outputiminus1 = ASDN(interpolatedi,irb_index=leveli.index).to(cpu)
    end.record()

    print("="*3, index, "="*3)
    print(f"Time passed (s) {start.elapsed_time(end) / 1000:.2f}")

    print(outputi.size())  
    print(outputiminus1.size()) 

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3063: UserWarning: Default upsampling behavior when mode=bicubic is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3103: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "


=== 0 ===
Time passed (s) 0.69
torch.Size([16, 3, 67, 67])
torch.Size([16, 3, 62, 62])
=== 1 ===
Time passed (s) 0.69
torch.Size([16, 3, 67, 67])
torch.Size([16, 3, 62, 62])
=== 2 ===
Time passed (s) 0.69
torch.Size([16, 3, 67, 67])
torch.Size([16, 3, 62, 62])
=== 3 ===
Time passed (s) 0.69
torch.Size([16, 3, 67, 67])
torch.Size([16, 3, 62, 62])


In [22]:
!nvidia-smi

Tue Dec  8 07:35:28 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    32W / 250W |  14449MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------